In [1]:
# Weak scaling test 2 
# 2 nodes
# 18 GB of data
from pyspark.sql import SparkSession
from functools import reduce
import json
import timeit
import pyspark.sql.functions as f 

spark_session = SparkSession\
        .builder\
        .master("spark://192.168.2.225:7077") \
        .appName("2json")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout","30s")\
        .config("spark.cores.max",2)\
        .getOrCreate()

In [2]:
%%time
start_time_0 = timeit.default_timer()

reddit = "hdfs://192.168.2.225:9000/reddit/"
range = [reddit + "RC_2012-01", reddit + "RC_2012-02"]
dataframes = map(lambda r: spark_session.read.json(r), range)
df = reduce(lambda df1, df2: df1.unionAll(df2), dataframes)

elapsed_0 = timeit.default_timer() - start_time_0



print("Time elapsed: ", elapsed_0, "s")

Time elapsed:  395.44475176697597 s
CPU times: user 92 ms, sys: 32.8 ms, total: 125 ms
Wall time: 6min 35s


In [3]:
df.count()

32365900

In [4]:
%%time
initial_start_time = timeit.default_timer()

if 'author_cakeday' in df.columns:
        current_df = df.drop('author_cakeday')
        
df = df.drop('author_flair_css_class','author_flair_text','can_gild','distinguished','edited','id','is_submitter','link_id','parent_id','permalink','retrieved_on','stickied','subreddit_id')

df = df.filter((df.body != '[deleted]'))


elapsed = timeit.default_timer() - initial_start_time



print("Time elapsed: ", elapsed, "s")

Time elapsed:  0.4420865389984101 s
CPU times: user 9.27 ms, sys: 1.81 ms, total: 11.1 ms
Wall time: 442 ms


In [5]:
%%time
start_time_1 = timeit.default_timer()

#df.select("author").show()
df.groupBy('subreddit').count().sort("count", ascending = False).show(10)

elapsed_1 = timeit.default_timer() - start_time_1

print("Time elapsed: ", elapsed_1,"s")

+-------------------+-------+
|          subreddit|  count|
+-------------------+-------+
|          AskReddit|3886207|
|              funny|1595579|
|               pics|1285955|
|            atheism| 958994|
|             gaming| 958909|
|           politics| 820115|
|                WTF| 793585|
|fffffffuuuuuuuuuuuu| 631135|
|              trees| 616926|
|               IAmA| 579836|
+-------------------+-------+
only showing top 10 rows

Time elapsed:  259.65715837001335 s
CPU times: user 67.4 ms, sys: 22 ms, total: 89.4 ms
Wall time: 4min 19s


In [6]:
%%time
start_time_2 = timeit.default_timer()

df = df.withColumn('wordCount', f.size(f.split(f.col('body'), ' ')))
#df.show()

df.groupBy('wordCount').count().sort("count", ascending = False).show(10)   

elapsed_2 = timeit.default_timer() - start_time_2


print("Time elapsed: ", elapsed_2,"s")

+---------+-------+
|wordCount|  count|
+---------+-------+
|        5|1176365|
|        6|1171644|
|        4|1135693|
|        7|1126744|
|        8|1072298|
|        1|1040225|
|        3|1037198|
|        9|1007129|
|       10| 945233|
|        2| 929350|
+---------+-------+
only showing top 10 rows

Time elapsed:  352.0260755979689 s
CPU times: user 82.1 ms, sys: 32.8 ms, total: 115 ms
Wall time: 5min 52s


In [7]:
%%time
start_time_3 = timeit.default_timer()

df_stats = df.select(f.mean(f.col('wordCount')).alias('mean')).collect()  #

mean = df_stats[0]['mean']

print("Average wordcount in comment: ", mean, " words.")


elapsed_3 = timeit.default_timer() - start_time_3
print("Time elapsed: ", elapsed_3,"s")

Average wordcount in comment:  32.29938768372247  words.
Time elapsed:  346.1760686310008 s
CPU times: user 92.8 ms, sys: 26.8 ms, total: 120 ms
Wall time: 5min 46s


In [8]:
total_elapsed = timeit.default_timer() - initial_start_time

print("Total time elapsed: ", total_elapsed, "s")

Total time elapsed:  958.3736905029509 s


In [9]:
spark_session.stop()